In [ ]:
!wget "https://drive.google.com/u/0/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download" -O english_python_data.txt

--2024-06-27 08:22:21--  https://drive.google.com/u/0/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download
Resolving drive.google.com (drive.google.com)... 74.125.197.102, 74.125.197.138, 74.125.197.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.197.102|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://drive.google.com/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download [following]
--2024-06-27 08:22:21--  https://drive.google.com/uc?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download [following]
--2024-06-27 08:22:21--  https://drive.usercontent.google.com/download?id=1rHb0FQ5z5ZpaY2HpyFGY6CeyDG0kTLoO&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.197.132, 2607:

In [ ]:
import re
import numpy as np

def load_data(filename, num_samples=None):
    with open(filename, 'r') as file:
        lines = file.read().split("\n\n")  # Split by double newlines for each example

    pairs = []
    for line in lines:
        if not line.strip():  # Skip empty blocks
            continue
        parts = line.split('\n', 1)
        if len(parts) == 2:
            english, python_code = parts
            pairs.append((english.strip(), python_code.strip()))

    if num_samples:
        pairs = pairs[:num_samples]

    return pairs

# Load a smaller subset of the data for testing
pairs = load_data("english_python_data.txt", num_samples=1000)

# Extract input and target texts
input_texts = [pair[0] for pair in pairs]
target_texts = ['\t' + pair[1] + '\n' for pair in pairs]  # Add start and end tokens

# Create vocabulary sets
input_characters = sorted(list(set(''.join(input_texts))))
target_characters = sorted(list(set(''.join(target_texts))))

# Create token index mappings
input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}

# Get max lengths
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

# Initialize data arrays
num_samples = len(pairs)
encoder_input_data = np.zeros((num_samples, max_encoder_seq_length, len(input_characters)), dtype='float32')
decoder_input_data = np.zeros((num_samples, max_decoder_seq_length, len(target_characters)), dtype='float32')
decoder_target_data = np.zeros((num_samples, max_decoder_seq_length, len(target_characters)), dtype='float32')

# Populate data arrays
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

# Reduce model complexity
latent_dim = 128

# Define the encoder
encoder_inputs = Input(shape=(None, len(input_characters)))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

# Define the decoder
decoder_inputs = Input(shape=(None, len(target_characters)))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(len(target_characters), activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the full model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Train the model with smaller batch size
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=32, epochs=10, validation_split=0.2)

# Save the full model
model.save('seq2seq_model.h5')




Epoch 1/10
25/25 [==============================] - 9s 198ms/step - loss: 0.3349 - val_loss: 0.2997
Epoch 2/10
25/25 [==============================] - 3s 106ms/step - loss: 0.3246 - val_loss: 0.2978
Epoch 3/10
25/25 [==============================] - 3s 107ms/step - loss: 0.3231 - val_loss: 0.2974
Epoch 4/10
25/25 [==============================] - 3s 123ms/step - loss: 0.3225 - val_loss: 0.2955
Epoch 5/10
25/25 [==============================] - 3s 126ms/step - loss: 0.3210 - val_loss: 0.2948
Epoch 6/10
25/25 [==============================] - 3s 107ms/step - loss: 0.3172 - val_loss: 0.2927
Epoch 7/10
25/25 [==============================] - 3s 106ms/step - loss: 0.3169 - val_loss: 0.2911
Epoch 8/10
25/25 [==============================] - 3s 106ms/step - loss: 0.3157 - val_loss: 0.2859
Epoch 9/10
25/25 [==============================] - 3s 125ms/step - loss: 0.3163 - val_loss: 0.2904
Epoch 10/10
25/25 [==============================] - 3s 118ms/step - loss: 0.3147 - val_loss: 0.2901

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.models import save_model

# Define inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)




# Save the models with compile information
save_model(encoder_model, 'encoder_model.h5')
save_model(decoder_model, 'decoder_model.h5')
# # Save the encoder model
# encoder_model.save('encoder_model.h5')

# # Save the decoder model
# decoder_model.save('decoder_model.h5')

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

<ipython-input-16-3222f03fa4e4>:21: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(encoder_model, 'encoder_model.h5')
<ipython-input-16-3222f03fa4e4>:22: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(decoder_model, 'decoder_model.h5')


In [ ]:
print(encoder_model.summary())
print(decoder_model.summary())

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 91)]        0         
                                                                 
 lstm (LSTM)                 [(None, 128),             112640    
                              (None, 128),                       
                              (None, 128)]                       
                                                                 
Total params: 112640 (440.00 KB)
Trainable params: 112640 (440.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, None, 103)]          0   

In [ ]:
# from tensorflow.keras.models import load_model

# # Load the models
# encoder_model = load_model('encoder_model.h5')
# decoder_model = load_model('decoder_model.h5')


from tensorflow.keras.optimizers import RMSprop

# Load the models
encoder_model = load_model('encoder_model.h5')
decoder_model = load_model('decoder_model.h5')

# Manually compile the models with explicit optimizer and loss
optimizer = RMSprop()
encoder_model.compile(optimizer=optimizer, loss='categorical_crossentropy')
decoder_model.compile(optimizer=optimizer, loss='categorical_crossentropy')


# Compile the models
encoder_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
decoder_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Ensure input_seq is in the correct shape
    input_seq = np.reshape(input_seq, (1, max_encoder_seq_length, len(input_characters)))

    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, len(target_characters)))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.0

    # Sampling loop for a batch of sequences
    stop_condition = False
    decoded_sentence = ''
    step = 0
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index.get(sampled_token_index, '')

        decoded_sentence += sampled_char
        step += 1

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length or
           step > max_decoder_seq_length):
            stop_condition = True

        target_seq = np.zeros((1, 1, len(target_characters)))
        target_seq[0, 0, sampled_token_index] = 1.0

        states_value = [h, c]

    return decoded_sentence


In [ ]:

# print("Welcome to the Conversational AI for Code Generation!")
# print("Type your prompt in English or 'exit' to quit.")

# while True:
#     user_prompt = input("Your prompt: ")
#     if user_prompt.lower() == 'exit':
#         print("Goodbye!")
#         break

#     input_seq = np.zeros((1, max_encoder_seq_length, len(input_characters)), dtype='float32')
#     for t, char in enumerate(user_prompt):
#         if char in input_token_index:
#             input_seq[0, t, input_token_index[char]] = 1.0

#     decoded_sentence = decode_sequence(input_seq)
#     print("Generated Code:\n")
#     print(decoded_sentence)
# Example user prompt
user_prompt = "write a python program to add two numbers"

# Prepare input sequence with correct shape
input_seq = np.zeros((max_encoder_seq_length, len(input_characters)), dtype='float32')
for t, char in enumerate(user_prompt):
    if char in input_token_index:
        input_seq[t, input_token_index[char]] = 1.0

# Reshape input_seq to match expected input shape for the model
input_seq = np.reshape(input_seq, (1, max_encoder_seq_length, len(input_characters)))

# Decode the sequence
decoded_sentence = decode_sequence(input_seq)
print("Generated Code:\n")
print(decoded_sentence)


Welcome to the Conversational AI for Code Generation!
Type your prompt in English or 'exit' to quit.
Your prompt: write a python program to add two numbers
1/1 [==============================] - 0s 30ms/step
Generated Code:

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

KeyboardInterrupt: Interrupted by user